# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».  
В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы.  
Нужно построить модель для задачи классификации, которая выберет подходящий тариф. При этом построенная модель должна быть с максимально большим значением accuracy (не менее 0.75).

## Изучение общей информации о данных

In [18]:
import pandas as pd
import numpy as np

from tqdm import tqdm
from pandas.core.common import random_state
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [19]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')

In [20]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


**Описание данных**   
   
- `сalls` — количество звонков,    
- `minutes` — суммарная длительность звонков в минутах,    
- `messages` — количество sms-сообщений,    
- `mb_used` — израсходованный интернет-трафик в Мб,   
- `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).  

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Данные хорошие, предобработка не требуется.

## Разбиение данных на выборки

Выведем сбалансированность целевого признака:

In [22]:
df['is_ultra'].mean()

0.30647168637212197

Создадим два датафрейма с обучающими признакми и целевым:

In [23]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

Разобьем данные: 60% - train set, 20% - validation set, 20% - test set

In [24]:
# Мне таким способом разделить датасет понравилось больше, но ниже сделаю как учили в уроке.
# features_train, features_valid, features_test = np.split(features.sample(frac=1, random_state=12345), [int(.6*len(features)), int(.8*len(features))])
# target_train, target_valid, target_test = np.split(target.sample(frac=1, random_state=12345), [int(.6*len(target)), int(.8*len(target))])

In [25]:
features_train, valid_test = train_test_split(features, test_size=0.4, random_state=12345)
features_valid, features_test = train_test_split(valid_test, test_size=0.5, random_state=12345)

In [26]:
target_train, valid_test = train_test_split(target, test_size=0.4, random_state=12345)
target_valid, target_test = train_test_split(valid_test, test_size=0.5, random_state=12345)

Посмотрим на разбиение выборок:

In [27]:
df_volumes = pd.DataFrame(
    {
     "Примечание": ['Доля обучающих признаков', 'Кол. обущаюших признаков', 'Доля целевых признаков', 'Кол. целевых признаков'],   
     "Обущающая_выборка": [round(len(features_train) / len(df), 2), len(features_train), round(len(target_train) / len(df), 2), len(target_train),],
     "Валидационная_выборка": [round(len(features_valid) / len(df), 2), len(features_valid), round(len(target_valid) / len(df), 2), len(target_valid),],
     "Тестовая_выборка": [round(len(features_test) / len(df), 2), len(features_test), round(len(target_test) / len(df), 2), len(target_test),],
    }
)
df_volumes

,Примечание,Обущающая_выборка,Валидационная_выборка,Тестовая_выборка
0,Доля обучающих признаков,0.6,0.2,0.2
1,Кол. обущаюших признаков,1928.0,643.0,643.0
2,Доля целевых признаков,0.6,0.2,0.2
3,Кол. целевых признаков,1928.0,643.0,643.0


## Исследование моделей

### Дерево решений

Обучим модель дерево решений:

In [28]:
%%time

best_model = None
best_result = 0
best_depth = 0 

for depth in range(1, 10):
  dtc_model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
  dtc_model.fit(features_train, target_train)
  predictions = dtc_model.predict(features_valid)
  result = accuracy_score(target_valid, predictions)
  if result > best_result:
    best_model = dtc_model
    best_result = 0
    best_depth = depth
    best_result = result

CPU times: user 80.9 ms, sys: 0 ns, total: 80.9 ms
Wall time: 83 ms


Выведем на экран долю правильных ответов и глубину лучшей модели дерева решений:

In [29]:
f"Доля правильных ответов: {best_result:.3f}, при глубине: {best_depth}"

'Доля правильных ответов: 0.785, при глубине: 3'

### Случайный лес

Обучим модель случайный лес:

In [37]:
%%time

best_model = None
best_result = 10000
best_est = 0
rfc_best_result = 0

for est in range(1, 30):
    for depth in range (1, 10):
        # инициализируйте модель RandomForestRegressor с параметрами random_state=12345, n_estimators=est и max_depth=depth
        rfc_model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) 
        rfc_model.fit(features_train, target_train) # обучите модель на тренировочной выборке
        predictions_valid = rfc_model.predict(features_valid) # получите предсказания модели на валидационной выборке
        rfc_result = rfc_model.score(features_valid, target_valid)
        if rfc_result > rfc_best_result:
            rfc_best_model = rfc_model
            rfc_best_result = rfc_result
            best_est = est
            rfc_best_depth = depth

CPU times: user 15.2 s, sys: 102 ms, total: 15.3 s
Wall time: 15.9 s


Выведем на экран долю правильных ответов, количеством оценщиков и глубиной лучшей модели случайный лес:

In [38]:
f"Доля правильных ответов: {rfc_best_result:.3f}, при количестве оценщиков: {best_est} и глубине: {rfc_best_depth}"

'Доля правильных ответов: 0.807, при количестве оценщиков: 26 и глубине: 7'

### Логистическая регрессия

Обучим модель логистическую регрессию:

In [32]:
%%time

lr_model = LogisticRegression(random_state=12345)
lr_model.fit(features_train, target_train)
result = lr_model.score(features_valid, target_valid)

CPU times: user 29.6 ms, sys: 0 ns, total: 29.6 ms
Wall time: 33.7 ms


Выведем на экран долю правильных ответов модели логистической регрессии:

In [33]:
f"Доля правильных ответов: {result:.3f}"

'Доля правильных ответов: 0.711'

### Вывод

1. Обучение модели на алгоритме Дерево решений: Время обучения: `122 ms`, Доля правильных ответов: `0.785`, при глубине: `3 `. 
2. Обучение модели на алгоритме Случайный лес: Время обучения: `36.7 s` Доля правильных ответов: `0.807`, при количестве оценщиков: `26` и глубине: `7`. 
3. Обучение модели на алгоритме Логистическся реггрессия: Время обучения: `34.6` ms Доля правильных ответов: `0.711`.

## Проверьте модель на тестовой выборке

In [39]:
df_scores = pd.DataFrame(
    {
     "Доля_правильных_ответов": ['Обущающая выборка выборка', 'Валидационная выборка', 'Тестовая выборка', 'Время моделирования'],   
     "Дерево_решений": [round(dtc_model.score(features_train, target_train), 2), 
                        round(dtc_model.score(features_valid, target_valid), 2), 
                        round(dtc_model.score(features_test, target_test), 2),
                        '~ 60 ms'],
     "Случайный_лес": [round(rfc_model.score(features_train, target_train), 2), 
                        round(rfc_model.score(features_valid, target_valid), 2), 
                        round(rfc_model.score(features_test, target_test), 2),
                       '~ 16 s'],
     "Логистическая_регрессия": [round(lr_model.score(features_train, target_train), 2), 
                        round(lr_model.score(features_valid, target_valid), 2), 
                        round(lr_model.score(features_test, target_test), 2),
                        '~ 25 ms'],          
    }
)
df_scores

,Доля_правильных_ответов,Дерево_решений,Случайный_лес,Логистическая_регрессия
0,Обущающая выборка выборка,0.88,0.88,0.71
1,Валидационная выборка,0.78,0.79,0.71
2,Тестовая выборка,0.78,0.81,0.68
3,Время моделирования,~ 60 ms,~ 16 s,~ 25 ms


## Проверка модели на адекватность

Проверить нашу модель на адекватность сравнив долю правильных ответов с алгоритмом Dummy:

In [48]:
%%time 

dummy_clf = DummyClassifier(random_state=12345)
dummy_clf.fit(features_train, target_train)

f"Доля правильных ответов Dummy: {dummy_clf.score(features_valid, target_valid):.2f}"

CPU times: user 2.26 ms, sys: 0 ns, total: 2.26 ms
Wall time: 8.7 ms


## Вывод

На тестовой выборке наилучший accuracy (доля правильных ответов) у алгоритма Случайный лес: `0.81`, у Дерева решений и Логистической регрессии: `0.78` и `0.68` соотственно.  
Важно ответить, что алгоритм Dummy показал accuracy: `0.71`, что означает алгоритм логистическая регрессия себя показывает очень плохо, им пользоваться нельзя.  

Сравнивая алгоритмы Дерево решений и Случайный лес, хоть и случайный лес показывает немного лучшее accuracy, но за много большее время обучения модели (`~ 60ms` и `~ 16s` соотвественно).  

Поэтому, для ответов на вопрос нашего исследования следуют выбрать алгоритм `Дерево решений`.